In [1]:
import os,sys,warnings,click
import h5py, pandas
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
np.set_printoptions(legacy='1.25')
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit,fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 24
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 6
mpl.rcParams['errorbar.capsize'] = 6
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
import util as yu
yu.flag_fast=False

enss=['b']
ens2full={'a24':'cA211.53.24','a':'cA2.09.48','b':'cB211.072.64','c':'cC211.060.80','d':'cD211.054.96','e':'cE211.044.112'}
ens2label={'a24':'A24','a':'A48','b':'B64','c':'C80','d':'D96','e':'E112'}
ens2a={'a24':0.0908,'a':0.0938,'b':0.07957,'c':0.06821,'d':0.05692,'e':0.04892} # fm
ens2N={'a24':24,'a':48,'b':64,'c':80,'d':96,'e':112}
ens2N_T={'a24':24*2,'a':48*2,'b':64*2,'c':80*2,'d':96*2,'e':112*2}

hbarc = 1/197.3
ens2aInv={ens:1/(ens2a[ens]*hbarc) for ens in enss} # MeV

def find_t_cloest(ens,t):
    return round(t/ens2a[ens])

In [2]:
stouts_global=[20]
stouts=stouts_global

ens2Njk={}
for ens in enss:
    path=f'/p/project1/ngff/li47/code/scratch/run/02_discNJN_1D_run3/{ens2full[ens]}/data_merge/data.h5'
    with h5py.File(path) as f:
        t=f['discNJN_j+;g{m,Dn};tl.h5']['notes'][:]
        projs=t[-1].decode().split('=')[-1][1:-1].split(',')
        t=f['j.h5/inserts'][:]
        inserts=[ele.decode() for ele in t]
        ens2Njk[ens]=len(f['N.h5/data/N_N'])
        
        inds_equal=[i for i,insert in enumerate(inserts) if insert[0]==insert[1]]
        inds_unequal=[i for i,insert in enumerate(inserts) if insert[0]!=insert[1]]
        
print(projs)
print(inserts)

['P0', 'Px', 'Py', 'Pz']
['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']


In [3]:
###
data={}
for ens in enss:
    path=f'/p/project1/ngff/li47/code/scratch/run/02_discNJN_1D_run3/{ens2full[ens]}/data_merge/data.h5'
    with h5py.File(path) as f:
        moms=f['N.h5/moms'][:]
        moms=[tuple(mom) for mom in moms]
        ind=moms.index((0,0,0))
        
        data[ens]=yu.jackknife(np.real(f['N.h5/data/N_N'][:,:,ind]))

propThreshold=0.1
# propThreshold=None

chi2Size=9
settings={}

func_C2pt_1st=lambda t,E0,c0: c0*np.exp(-E0*t)
func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C2pt_3st=lambda t,E0,c0,dE1,rc1,dE2,rc2: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t) + rc2*np.exp(-dE2*t))
func_mEff_1st=lambda t,E0: np.log(func_C2pt_1st(t,E0,1)/func_C2pt_1st(t+1,E0,1))
func_mEff_2st=lambda t,E0,dE1,rc1: np.log(func_C2pt_2st(t,E0,1,dE1,rc1)/func_C2pt_2st(t+1,E0,1,dE1,rc1))
func_mEff_3st=lambda t,E0,dE1,rc1,dE2,rc2: np.log(func_C2pt_3st(t,E0,1,dE1,rc1,dE2,rc2)/func_C2pt_3st(t+1,E0,1,dE1,rc1,dE2,rc2))

def run(ens,figname=None):
    corrQ=True; meffQ=True
    xunit=ens2a[ens]; yunit=ens2aInv[ens]/1000
    fig, axd = plt.subplot_mosaic([['f1','f1','f1'],['f2','f2','f3']],figsize=(24,10))
    (ax1,ax2,ax3)=(axd[key] for key in ['f1','f2','f3'])
    # if meffQ:
    #     fig.suptitle('Correlated fit to meff' if corrQ else 'Uncorrelated fit to meff',fontsize=44)
    # else:
    #     fig.suptitle('Correlated fit to C2pt' if corrQ else 'Uncorrelated fit to C2pt',fontsize=44)
    
    ax1.set_xlabel(r'$t$ [fm]')
    ax2.set_xlabel(r'$t_{\mathrm{min}}$ [fm]')
    ax3.set_xlabel(r'$t_{\mathrm{min}}$ [fm]')
    ax1.set_ylabel(r'$m_N^{\mathrm{eff}}$ [GeV]')
    ax2.set_ylabel(r'$m_N$ [GeV]')
    ax3.set_ylabel(r'$E_1$ [GeV]')
    ax1.set_ylim(settings['ylim1'])
    ax2.set_ylim(settings['ylim2'])
    ax3.set_ylim(settings['ylim3'])
    ax1.set_xlim(settings['xlim1'])
    ax2.set_xlim(settings['xlim2'])
    ax3.set_xlim(settings['xlim3'])
    
    mN_exp=0.938; mp_exp,mn_exp=(0.93827,0.93957)
    ax1.axhline(y=mN_exp,color='black',linestyle = '--', marker='')
    ax2.axhline(y=mN_exp,color='black',linestyle = '--', marker='', label=r'$m_N^{\mathrm{exp}}=$'+'%0.3f'%mN_exp)
    C2pt_jk=data[ens]
    C2pt_mean,C2pt_err=yu.jackme(C2pt_jk)
    C2pt_rela=np.abs(C2pt_err/C2pt_mean)
    func=lambda C2pt: np.log(C2pt/np.roll(C2pt,-1,axis=0))
    mEff_jk=yu.jackmap(func,C2pt_jk)
    (mEff_mean,mEff_err)=yu.jackme(mEff_jk)
    mEff_rela=np.abs(mEff_err/mEff_mean)
    temp=[(i,rela) for i,rela in enumerate(mEff_rela if meffQ else C2pt_rela) if rela>0.2 and i!=0]
    fitmax=temp[0][0]-1 if len(temp)!=0 else len(C2pt_mean)-1
    
    tmin=1; tmax=fitmax+1
    plt_x=np.arange(tmin,tmax)*xunit; plt_y=mEff_mean[tmin:tmax]*yunit; plt_err=mEff_err[tmin:tmax]*yunit
    ax1.errorbar(plt_x,plt_y,plt_err,color='black',fmt='s')

    pars0_initial=[0.4,0.5,2,0.8,1] if meffQ else [0.4,1e-8,0.5,2,0.8,1]
    DNpar=1 if meffQ else 0
    
    fits_all=[]
    # 1st fits
    color='r'
    fitmins=settings['fitmins_1st']
    pars0=pars0_initial[:2-DNpar]
    fits=[]
    for fitmin in fitmins:
        tList=np.arange(fitmin,fitmax,2)
        def fitfunc(pars):
            if meffQ:
                return func_mEff_1st(tList,*pars)
            return func_C2pt_1st(tList,*pars)
        y_jk=mEff_jk[:,tList] if meffQ else C2pt_jk[:,tList]
        pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,y_jk,pars0,mask=None if corrQ else 'uncorrelated')
        pars0=np.mean(pars_jk,axis=0)
        fits.append([fitmin,pars_jk,chi2_jk,Ndof])
        fits_all.append([('1st',fitmin),pars_jk[:,:1],chi2_jk,Ndof])
        
    pars_jk,props_jk=yu.jackMA(fits)
    props_mean=np.mean(props_jk,axis=0)
    ind_mpf=np.argmax(np.mean(props_jk,axis=0))    
    pars_mean,pars_err=yu.jackme(pars_jk)
    pars0=pars_mean
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xunit; plt_y=pars_mean[0]*yunit; plt_err=pars_err[0]*yunit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2,label=r'$m_N^{\mathrm{1st}}=$'+yu.un2str(plt_y,plt_err))
    for i,fit in enumerate(fits):
        fitmin,pars_jk,chi2_jk,Ndof=fit; prop=props_mean[i]
        (pars_mean,pars_err)=yu.jackme(pars_jk)
        chi2R=np.mean(chi2_jk)/Ndof
        showQ = i==ind_mpf if propThreshold is None else prop>propThreshold
        
        plt_x=fitmin*xunit; plt_y=pars_mean[0]*yunit; plt_err=pars_err[0]*yunit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white' if showQ else None)
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        if propThreshold is not None and prop>propThreshold:
            ax2.annotate(f"{int(prop*100)}%",(plt_x,plt_y+plt_err+chi2_shift*0.5),color=color,size=chi2Size,ha='center')
            
    # 2st fits
    color='g'
    fitmins=settings['fitmins_2st']
    pars0=np.hstack([pars0,pars0_initial[2-DNpar:4-DNpar]])
    fits=[]
    for fitmin in fitmins:
        # print(2,fitmin)
        tList=np.arange(fitmin,fitmax,2)
        def fitfunc(pars):
            if meffQ:
                return func_mEff_2st(tList,*pars)
            return func_C2pt_2st(tList,*pars)
        y_jk=mEff_jk[:,tList] if meffQ else C2pt_jk[:,tList]
        pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,y_jk,pars0,mask=None if corrQ else 'uncorrelated')
        pars0=np.mean(pars_jk,axis=0)
        fits.append([fitmin,pars_jk,chi2_jk,Ndof])
        fits_all.append([('2st',fitmin),pars_jk[:,:1],chi2_jk,Ndof])
    pars_jk,props_jk=yu.jackMA(fits)
    props_mean=np.mean(props_jk,axis=0)
    res=pars_jk.copy()
    ind_mpf=np.argmax(np.mean(props_jk,axis=0))    
    pars0=yu.jackme(pars_jk)[0]
    pars_jk[:,1]=pars_jk[:,0]+pars_jk[:,2-DNpar]
    pars_mean,pars_err=yu.jackme(pars_jk)
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xunit; plt_y=pars_mean[0]*yunit; plt_err=pars_err[0]*yunit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$m_N^{\mathrm{2st}}=$'+yu.un2str(plt_y,plt_err))
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xunit; plt_y=pars_mean[1]*yunit; plt_err=pars_err[1]*yunit
    ax3.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$E_1^{\mathrm{2st}}=$'+yu.un2str(plt_y,plt_err))
    for i,fit in enumerate(fits):
        fitmin,pars_jk,chi2_jk,Ndof=fit; prop=props_mean[i]
        pars_jk[:,1]=pars_jk[:,0]+pars_jk[:,2-DNpar]
        (pars_mean,pars_err)=yu.jackme(pars_jk)
        chi2R=np.mean(chi2_jk)/Ndof
        showQ = i==ind_mpf if propThreshold is None else prop>propThreshold
        
        plt_x=fitmin*xunit; plt_y=pars_mean[0]*yunit; plt_err=pars_err[0]*yunit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white' if showQ else None)
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        if propThreshold is not None and prop>propThreshold:
            ax2.annotate(f"{int(prop*100)}%",(plt_x,plt_y+plt_err+chi2_shift*0.5),color=color,size=chi2Size,ha='center')
        
        plt_x=fitmin*xunit; plt_y=pars_mean[1]*yunit; plt_err=pars_err[1]*yunit
        ax3.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white' if showQ else None)
        ylim=ax3.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax3.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        if propThreshold is not None and prop>propThreshold:
            ax3.annotate(f"{int(prop*100)}%",(plt_x,plt_y+plt_err+chi2_shift*0.5),color=color,size=chi2Size,ha='center')

    # 3st fits
    color='b'
    fitmins=settings['fitmins_3st']
    pars0=np.hstack([pars0,pars0_initial[4-DNpar:6-DNpar]])
    if ens=='c' and (corrQ,meffQ)==(False,False):
        pars0=[3.25069715e-01, 1.88384811e-09, 1.78883939e-01, 6.35351339e-01, 6.98775484e-01, 4.58702896e+01]
    # elif ens=='d' and (corrQ,meffQ)==(False,False):
    #     pars=[2.72824764e-01, 3.72721072e-10, 1.84246641e-01, 7.65383428e-01, 6.98775484e-01, 4.58702896e+01]
    fits=[]
    for fitmin in fitmins:
        # print(3,fitmin)
        tList=np.arange(fitmin,fitmax,2)
        def fitfunc(pars):
            if meffQ:
                return func_mEff_3st(tList,*pars)
            return func_C2pt_3st(tList,*pars)
        y_jk=mEff_jk[:,tList] if meffQ else C2pt_jk[:,tList]
        pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,y_jk,pars0,mask=None if corrQ else 'uncorrelated')
        pars0=np.mean(pars_jk,axis=0)
        fits.append([fitmin,pars_jk,chi2_jk,Ndof])
        fits_all.append([('3st',fitmin),pars_jk[:,:1],chi2_jk,Ndof])
    pars_jk,props_jk=yu.jackMA(fits)
    props_mean=np.mean(props_jk,axis=0)
    ind_mpf=np.argmax(np.mean(props_jk,axis=0))    
    pars0=yu.jackme(pars_jk)[0]
    # print(pars0)
    pars_jk[:,1]=pars_jk[:,0]+pars_jk[:,2-DNpar]
    pars_mean,pars_err=yu.jackme(pars_jk)
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xunit; plt_y=pars_mean[0]*yunit; plt_err=pars_err[0]*yunit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$m_N^{\mathrm{3st}}=$'+yu.un2str(plt_y,plt_err))
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xunit; plt_y=pars_mean[1]*yunit; plt_err=pars_err[1]*yunit
    ax3.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$E_1^{\mathrm{3st}}=$'+yu.un2str(plt_y,plt_err))    
    for i,fit in enumerate(fits):
        fitmin,pars_jk,chi2_jk,Ndof=fit; prop=props_mean[i]
        pars_jk[:,1]=pars_jk[:,0]+pars_jk[:,2-DNpar]
        (pars_mean,pars_err)=yu.jackme(pars_jk)
        chi2R=np.mean(chi2_jk)/Ndof
        showQ = i==ind_mpf if propThreshold is None else prop>propThreshold
        
        plt_x=fitmin*xunit; plt_y=pars_mean[0]*yunit; plt_err=pars_err[0]*yunit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white' if showQ else None)
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        if propThreshold is not None and prop>propThreshold:
            ax2.annotate(f"{int(prop*100)}%",(plt_x,plt_y+plt_err+chi2_shift*0.5),color=color,size=chi2Size,ha='center')
        
        plt_x=fitmin*xunit; plt_y=pars_mean[1]*yunit; plt_err=pars_err[1]*yunit
        ax3.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white' if showQ else None)
        ylim=ax3.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax3.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center') 
        if propThreshold is not None and prop>propThreshold:
            ax3.annotate(f"{int(prop*100)}%",(plt_x,plt_y+plt_err+chi2_shift*0.5),color=color,size=chi2Size,ha='center')
        
    color='orange'
    pars_jk,props_jk=yu.jackMA(fits_all)
    ind_mpf=np.argmax(np.mean(props_jk,axis=0))
    pars_mean,pars_err=yu.jackme(pars_jk)
    plt_x=settings['xlim2']; plt_y=pars_mean[0]*yunit; plt_err=pars_err[0]*yunit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$m_N^{\mathrm{nst}}=$'+yu.un2str(plt_y,plt_err) + f'; MPF: {fits_all[ind_mpf][0][0]}')    
    
    ax2.legend(loc=(0.6,0.5),fontsize=12)
    ax3.legend(fontsize=12)
    
    plt.tight_layout()
    # plt.savefig(figname)
    plt.close()
    return res

res_c2ptN0={}
for ens in enss[:]:
    if ens=='b':
        settings={
            'fitmins_1st':range(8,24+1),
            'fitmins_2st':range(1,10+1),
            'fitmins_3st':range(1,4+1),
            'ylim1':[0.6,1.7],
            'ylim1':[0.88,1.08],
            'ylim2':[0.85,1.1],
            'ylim3':[0.85,3.0],
            'xlim1':[0,2.7],
            'xlim2':[0,2.2],
            'xlim3':[0,1.1],
        }
    elif ens=='c':
        settings={
            'fitmins_1st':range(8,29+1),
            'fitmins_2st':range(1,17+1),
            'fitmins_3st':range(1,7+1),
            'ylim1':[0.6,1.7],
            'ylim1':[0.88,1.08],
            'ylim2':[0.85,1.1],
            'ylim3':[0.85,3.0],
            'xlim1':[0,2.7],
            'xlim2':[0,2.2],
            'xlim3':[0,1.1],
        }
    elif ens=='d':
        settings={
            'fitmins_1st':range(8,34+1),
            'fitmins_2st':range(1,20+1),
            'fitmins_3st':range(1,6+1),
            'ylim1':[0.6,1.7],
            'ylim1':[0.88,1.08],
            'ylim2':[0.85,1.1],
            'ylim3':[0.85,3.0],
            'xlim1':[0,2.7],
            'xlim2':[0,2.2],
            'xlim3':[0,1.1],
        }
    res_c2ptN0[ens]=run(ens,'')
    
ens2mN={}
for ens in enss:
    ens2mN[ens]=res_c2ptN0[ens][:,0]

In [24]:
###
import sympy as sp
from sympy import sqrt
from itertools import permutations

id=np.eye(4)
g1=np.array([[0, 0, 0, 1j],
            [0, 0, 1j, 0],
            [0, -1j, 0, 0],
            [-1j, 0, 0, 0]])

g2=np.array([[0, 0, 0, 1],
            [0, 0, -1, 0],
            [0, -1, 0, 0],
            [1, 0, 0, 0]])

g3=np.array([[0, 0, 1j, 0],
            [0, 0, 0, -1j],
            [-1j, 0, 0, 0],
            [0, 1j, 0, 0]])

g4=np.array([[1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, -1, 0],
            [0, 0, 0, -1]])

g5 = g1@g2@g3@g4
gm = np.array([g1, g2, g3, g4])
sgm = np.array([[(gm[mu]@gm[nu] - gm[nu]@gm[mu])/2 for nu in range(4)] for mu in range(4)])

G0 = (id + g4) / 4
G1 = 1j * g5 @ g1 @ G0
G2 = 1j * g5 @ g2 @ G0
G3 = 1j * g5 @ g3 @ G0
G = [G1, G2, G3, G0]

insert2ind={'x':0,'y':1,'z':2,'t':3}
def ME2FF(m,pvec,pvec1,proj,insert):
    Gn={'P0':G0,'Px':G1,'Py':G2,'Pz':G3}[proj]
    mu,nu=insert
    mu=insert2ind[mu]; nu=insert2ind[nu]
    
    px,py,pz=pvec
    p1x,p1y,p1z=pvec1
    
    if m==sp.symbols('m'):
        pt=sp.symbols('pt')
        p1t=sp.symbols('p1t')
    else:
        pt=1j*np.sqrt(px**2+py**2+pz**2+m**2)
        p1t=1j*np.sqrt(p1x**2+p1y**2+p1z**2+m**2)
        
    p=np.array([px,py,pz,pt])
    p1=np.array([p1x,p1y,p1z,p1t])

    pS=np.sum(gm*p[:,None,None],axis=0)
    p1S=np.sum(gm*p1[:,None,None],axis=0)
    Px, Py, Pz, Pt = p + p1
    qx, qy, qz, qt = p - p1
    P=np.array([Px,Py,Pz,Pt])
    q=np.array([qx,qy,qz,qt])
    Q2 = -2*m**2 - 2*p1.dot(p)
    
    #==============================
    factorA= 1j; factorB= -1j; factorC=1
    factorBase=sp.symbols('K')/(4*m**2); factorSgm=1
    
    if m!=sp.symbols('m'):
        xE=pt/1j; xE1=p1t/1j
        factorBase=1/np.sqrt(2*xE1*(xE1+m)*2*xE*(xE+m))
    
    la=(gm[mu]*P[nu]/2+gm[nu]*P[mu]/2)/2-(np.sum(gm*P[:,None,None]/2,axis=0))*id[mu,nu]/4
    lb=(1j/(2*m))*((np.einsum('rab,r->ab',sgm[mu],q)*P[nu]/2+np.einsum('rab,r->ab',sgm[nu],q)*P[mu]/2)/2-np.einsum('srab,r,s->ab',sgm,q,P/2)*id[mu,nu]/4)*factorSgm
    lc=(id/m)*(q[mu]*q[nu]-Q2/4*id[mu,nu])
    
    res=np.array([factorBase*factor*np.trace(Gn@(-1j*p1S+m*id)@Lambda@(-1j*pS+m*id)) for Lambda,factor in zip([la,lb,lc],[factorA,factorB,factorC])])
    
    if m==sp.symbols('m'):
        xE = sp.symbols('E')
        xE1 = sp.symbols('E1')
        xE1=xE=sqrt(px**2+py**2+pz**2+m**2)
        for t in res:
            # t=t.subs({p1x:0,p1y:0,p1z:0,p1t:1j*m,pt:1j*xE})
            # t=t.subs({px:sqrt(xE**2-m**2-py**2-pz**2)})            
            t=t.subs({p1t:1j*xE1,pt:1j*xE})
            if px==p1x and py==p1y and pz==p1z:
                t=t.subs({sp.symbols('K'):(4*m**2)/(2*xE*(xE+m))})
            t=sp.expand(sp.sympify(t))
            print(t)
        print()
        return
    
    return res

def nonzeroQ(mom,proj,insert):
    n1vec=np.array(mom[:3]); nqvec=np.array(mom[3:6])
    nvec=n1vec+nqvec
    
    m=938/ens2aInv[ens]; L=ens2N[ens]
    pvec=nvec*(2*np.pi/L); p1vec=n1vec*(2*np.pi/L)
    
    res=ME2FF(m,pvec,p1vec,proj,insert)
    tr=np.sum(np.abs(np.real(res))); ti=np.sum(np.abs(np.imag(res)))
    threshold=1e-8
    return (tr>threshold,ti>threshold)

def rotateMPI(rot,mom,proj,insert):
    sx,sy,sz,xyz=rot; signs=[sx,sy,sz,1]
    ix,iy,iz=xyz; iix,iiy,iiz=tuple([ix,iy,iz].index(i) for i in range(3))
    xyzt=['x','y','z','t']
    xyzt2={'x':xyzt[ix],'y':xyzt[iy],'z':xyzt[iz],'t':'t'}
    
    mom1=[sx*mom[iix],sy*mom[iiy],sz*mom[iiz],sx*mom[iix+3],sy*mom[iiy+3],sz*mom[iiz+3]]
    proj1='P0' if proj=='P0' else f'P{xyzt2[proj[1]]}'
    insert1=f'{xyzt2[insert[0]]}{xyzt2[insert[1]]}'
    insert1=insert1 if insert1 in inserts else insert1[1]+insert1[0]
    return [mom1,proj1,insert1]

def sortFunc(mpi):
    return ''.join(mpi)

def useQ(mom,proj,insert):
    r,i=nonzeroQ(mom,proj,insert)
    if (r,i)==(False,False):
        return (False,False)
    if insert == 'tt': # traceless makes tt=-xx-yy-zz
        return (False,False)
    
    elements=[(sx,sy,sz,xyz) for sx in [1,-1] for sy in [1,-1] for sz in [1,-1] for xyz in permutations([0, 1, 2], 3)]
    mpis=[rotateMPI(e,mom,proj,insert) for e in elements]
    pis=[('_'.join([str(e) for e in m]),p,i) for m,p,i in mpis if np.all(list(m[3:])==mom[3:])]
    pis=list(set(pis))
    pis.sort(key=sortFunc)
    if ('_'.join([str(e) for e in mom]),proj,insert) != pis[-1]:
        return (False,False)
    return (r,i)

def useQ_noAvg(mom,proj,insert):
    r,i=nonzeroQ(mom,proj,insert)
    if (r,i)==(False,False):
        return (False,False)
    if insert == 'tt': # traceless makes tt=-xx-yy-zz
        return (False,False)
    
    return (r,i)

ME2FF(sp.symbols('m'),sp.symbols('px py pz'),sp.symbols('p1x p1y p1z'),'P0','tt')

# ME2FF(1,[0,0,0],[0,0,0],'P0','tt')

# nonzeroQ([0,0,0,0,0,0],'P0','tt')

# useQ([0,0,0,0,0,0],'P0','zz')

# useQ([1,0,0,0,0,0],'P0','xx')
###

-0.375*K*m - 0.375*K*sqrt(m**2 + px**2 + py**2 + pz**2) - 0.03125*K*p1x**2/m - 0.0625*K*p1x*px/m - 0.03125*K*p1y**2/m - 0.0625*K*p1y*py/m - 0.03125*K*p1z**2/m - 0.0625*K*p1z*pz/m - 0.40625*K*px**2/m - 0.40625*K*py**2/m - 0.40625*K*pz**2/m - 0.03125*K*p1x**2*sqrt(m**2 + px**2 + py**2 + pz**2)/m**2 - 0.25*K*p1x*px*sqrt(m**2 + px**2 + py**2 + pz**2)/m**2 - 0.03125*K*p1y**2*sqrt(m**2 + px**2 + py**2 + pz**2)/m**2 - 0.25*K*p1y*py*sqrt(m**2 + px**2 + py**2 + pz**2)/m**2 - 0.03125*K*p1z**2*sqrt(m**2 + px**2 + py**2 + pz**2)/m**2 - 0.25*K*p1z*pz*sqrt(m**2 + px**2 + py**2 + pz**2)/m**2 - 0.21875*K*px**2*sqrt(m**2 + px**2 + py**2 + pz**2)/m**2 - 0.21875*K*py**2*sqrt(m**2 + px**2 + py**2 + pz**2)/m**2 - 0.21875*K*pz**2*sqrt(m**2 + px**2 + py**2 + pz**2)/m**2
0.09375*K*p1x**2/m - 0.1875*K*p1x*px/m + 0.09375*K*p1y**2/m - 0.1875*K*p1y*py/m + 0.09375*K*p1z**2/m - 0.1875*K*p1z*pz/m + 0.09375*K*px**2/m + 0.09375*K*py**2/m + 0.09375*K*pz**2/m + 0.09375*K*p1x**2*sqrt(m**2 + px**2 + py**2 + pz**2)/m**2 - 

In [ ]:
# ME2FF(sp.symbols('m'),sp.symbols('px py pz'),sp.symbols('px py pz'),'P0','tz') # i pz
ME2FF(sp.symbols('m'),sp.symbols('px py pz'),sp.symbols('px py pz'),'P0','tt') # -m2/E *3/4 - (E2-m2)/E

-0.75*m**3/(m**2 + m*sqrt(m**2 + px**2 + py**2 + pz**2) + px**2 + py**2 + pz**2) - 0.75*m**2*sqrt(m**2 + px**2 + py**2 + pz**2)/(m**2 + m*sqrt(m**2 + px**2 + py**2 + pz**2) + px**2 + py**2 + pz**2) - 1.0*m*px**2/(m**2 + m*sqrt(m**2 + px**2 + py**2 + pz**2) + px**2 + py**2 + pz**2) - 1.0*m*py**2/(m**2 + m*sqrt(m**2 + px**2 + py**2 + pz**2) + px**2 + py**2 + pz**2) - 1.0*m*pz**2/(m**2 + m*sqrt(m**2 + px**2 + py**2 + pz**2) + px**2 + py**2 + pz**2) - 1.0*px**2*sqrt(m**2 + px**2 + py**2 + pz**2)/(m**2 + m*sqrt(m**2 + px**2 + py**2 + pz**2) + px**2 + py**2 + pz**2) - 1.0*py**2*sqrt(m**2 + px**2 + py**2 + pz**2)/(m**2 + m*sqrt(m**2 + px**2 + py**2 + pz**2) + px**2 + py**2 + pz**2) - 1.0*pz**2*sqrt(m**2 + px**2 + py**2 + pz**2)/(m**2 + m*sqrt(m**2 + px**2 + py**2 + pz**2) + px**2 + py**2 + pz**2)
0
0



In [144]:
# SVD

from scipy.linalg import sqrtm
funcs_ri=[np.real,np.imag]

name='_unequal'

def standarizeMom(mom):
    t=np.abs(mom)
    t.sort()
    return t

FFs=['A20','B20','C20']
def run(ens):
    inpath=f'/p/project1/ngff/li47/code/scratch/run/02_discNJN_1D_run3/{ens2full[ens]}_backup/data_merge/data.h5'
    outpath=f'/p/project1/ngff/li47/code/scratch/run/02_discNJN_1D_run3/{ens2full[ens]}_backup/data_merge/data_SVD{name}.h5'
    with h5py.File(inpath) as f, h5py.File(outpath,'w') as fw:
        moms_N=f['N.h5/moms'][:]
        dic_N={}
        for i,mom in enumerate(moms_N):
            dic_N[tuple(mom)]=i
        cN=f['N.h5/data/N_N'][:] # jackknife done in cNa, cNb
        
        file='discNJN_jg;stout.h5'
        moms_3pt=f[file]['moms'][:]
        keys=list(f[f'{file}/data'].keys()); keys.sort()
        keys=['N_N_jg;stout20_6']
        for ikey,key in enumerate(keys):
            # if ikey<294:
            #     continue
            _,_,j,tf=key.split('_')
            if j=='jv1':
                continue
            # if j!='jq;stout10':
            #     continue
            tf=int(tf)
            # print(ens,f'{ikey}/{len(keys)}',key,end='              \r')
            data=f[f'{file}/data'][key][:]
            
            for imom,mom in enumerate(moms_3pt):
                if not np.all(mom==[0,0,0,0,1,1]):
                    continue
                mom_str='_'.join([str(ele) for ele in mom])
                # print(imom,len(moms_3pt),mom_str)
                pa=mom[:3]; q=mom[3:6]; pb=pa+q
                
                cNa=cN[:,:,dic_N[tuple(standarizeMom(pa))]]
                cNb=cN[:,:,dic_N[tuple(standarizeMom(pb))]]
                cNa=yu.jackknife(cNa); cNb=yu.jackknife(cNb)
                
                Njk=len(cNa)
                
                L=ens2N[ens]
                n1vec=np.array(mom[:3]); nqvec=np.array(mom[3:6])
                nvec=n1vec+nqvec
                pvec=nvec*(2*np.pi/L); p1vec=n1vec*(2*np.pi/L)
                qvec=nqvec*(2*np.pi/L)
                
                xE_jk=np.sqrt(pvec.dot(pvec)+ens2mN[ens]**2)
                xE1_jk=np.sqrt(p1vec.dot(p1vec)+ens2mN[ens]**2)
                Q2_jk=(qvec.dot(qvec) - (xE_jk-xE1_jk)**2 )
                Q2=np.mean(Q2_jk)
                
                c3pt=data[:,:,imom,:,:]
                c3pt=yu.jackknife(c3pt)
                print(f'{j};mom={mom};ts=6,tins=3;i_jk=0')
                ratio=c3pt/np.sqrt(
                        cNa[:,tf:tf+1]*cNb[:,tf:tf+1]*\
                        cNa[:,:tf+1][:,::-1]/cNa[:,:tf+1]*\
                        cNb[:,:tf+1]/cNb[:,:tf+1][:,::-1]
                )[:,:,None,None]
                
                pirs=[(proj,insert,ri) for proj in projs for insert in inserts for ri in [0,1] if insert[0]!=insert[1] and useQ(mom,proj,insert)[ri]]
                # pirs=[(proj,insert,ri) for proj in projs for insert in inserts for ri in [0,1] if useQ(mom,proj,insert)[ri]]
                G=np.array([[funcs_ri[ri](ME2FF(m,pvec,p1vec,proj,insert)) for proj,insert,ri in pirs] for m in ens2mN[ens]])
                
                print('components:')
                print(pirs)
                print('decomposition w/o dividing error:')
                print(G[0])
                if len(G[0])==0:
                    rank=0
                else:
                    U, S, VT = np.linalg.svd(G[0])
                    tol = 1e-10
                    rank = np.sum(S > tol)
                if rank == 3 or (rank==1 and np.all(q==[0,0,0])):
                    M_all=np.transpose([funcs_ri[ri](ratio[:,:,projs.index(proj),inserts.index(insert)]) for proj,insert,ri in pirs],[1,2,0])
                    # print(M_all.shape)
                    print('matrix elements:')
                    print(M_all[0,3])
                    t=np.zeros([Njk,tf+1,rank])
                    for tc in range(tf+1):
                        M=M_all[:,tc]
                        cov=yu.jackmec(M)[-1]
                        cov=np.diag(np.diag(cov))
                        covI=np.linalg.inv(cov)
                        covIsq = sqrtm(covI)
                        def get(g,m):
                            gt=covIsq@g
                            u,s,vT=np.linalg.svd(gt)
                            sI=np.zeros(gt.T.shape)
                            np.fill_diagonal(sI,1/s)
                            return vT.T@sI@(u.T)@covIsq@m
                        F=np.array([get(g[:,:rank],m) for g,m in zip(G,M)])
                        
                        if tc==3:
                            print('inverse err:')
                            print(np.diag(covIsq))
                            print('decomposition divided by errors:')
                            print(covIsq@G[0])
                            print('combinations')
                            gt=covIsq@G[0]
                            u,s,vT=np.linalg.svd(gt)
                            sI=np.zeros(gt.T.shape)
                            np.fill_diagonal(sI,1/s)
                            tt=vT.T@sI@(u.T)@covIsq
                            print(tt[1])
                            print(M_all[0,3]*tt[1])
                        
                        t[:,tc,:]=F
                    for i in range(t.shape[-1]):
                        FF=FFs[i]
                        key=f'{FF}_{j}/{mom_str}/{tf}'
                        if key in fw:
                            del fw[key]
                        fw.create_dataset(key,data=t[:,:,i])
                    print('A20,B20,C20:')
                    print(t[0,3])
                    print(t[0,:])
                    print('mean,err')
                    print(t.shape)
                    print(yu.jackme(t[:,3]))
        
run('b')

jg;stout20;mom=[0 0 0 0 1 1];ts=6,tins=3;i_jk=0
components:
[('P0', 'tz', 1), ('P0', 'yz', 0), ('Px', 'tz', 0), ('Pz', 'tx', 0), ('Pz', 'xy', 1), ('Pz', 'xz', 1)]
decomposition w/o dividing error:
[[ 0.04833674 -0.00156184  0.00624734]
 [ 0.00604694 -0.00019539  0.02496932]
 [ 0.02416837  0.02416837  0.        ]
 [-0.02416837 -0.02416837  0.        ]
 [ 0.00302347  0.00302347  0.        ]
 [ 0.00302347  0.00302347  0.        ]]
matrix elements:
[ 0.01355997 -0.0093984   0.00600327 -0.00628641 -0.00014505  0.00015421]
inverse err:
[ 266.71847767  178.07622084 1676.47857558 1608.83858789 1646.35637233
 1648.28036117]
decomposition divided by errors:
[[ 1.28923015e+01 -4.16570270e-01  1.66628108e+00]
 [ 1.07681674e+00 -3.47936200e-02  4.44644143e+00]
 [ 4.05177538e+01  4.05177538e+01  0.00000000e+00]
 [-3.88830056e+01 -3.88830056e+01  0.00000000e+00]
 [ 4.97771148e+00  4.97771148e+00  0.00000000e+00]
 [ 4.98352860e+00  4.98352860e+00  0.00000000e+00]]
combinations
[-20.68819738   5.176201

In [101]:
# SVD noAvg

from scipy.linalg import sqrtm
funcs_ri=[np.real,np.imag]

name='_unequal'

path='/p/project1/ngff/li47/code/glwc2/project2/02_discNJN_1D/dataPrepare/cB211.072.64/data_aux/cfgs_run'
with open(path,'r') as f:
    cfgs=f.read().splitlines()
print(len(cfgs))

inpath=f'/p/project1/ngff/li47/code/scratch/run/02_discNJN_1D_run2/cB211.072.64/data_avgsrc/'
flag=True
data=[]
for icfg,cfg in enumerate(cfgs):
    print(f'{icfg}/{len(cfgs)}',end='           \r')
    infile=f'{inpath}/{cfg}/discNJN_jg;stout.h5'
    with h5py.File(infile) as f:
        if flag:
            moms=f['moms'][:]
            dic={}
            for i,mom in enumerate(moms):
                dic[tuple(mom)]=i
            
            moms_target=[list(mom) for mom in moms if mom[3]**2+mom[4]**2+mom[5]**2==2]
            inds=[dic[tuple(mom)] for mom in moms_target]
            flag=False       
            
        # print(f['data']['N_N_jg;stout20_6'])
        t=f['data']['N_N_jg;stout20_6'][:,inds,:,:]
        data.append(t)
    # break
data=np.array(data)
c3pt=yu.jackknife(data)
c3pt_0avg=c3pt.copy()
print(data.shape)

inpath=f'/p/project1/ngff/li47/code/scratch/run/02_discNJN_1D_run2/cB211.072.64/data_avgmore/'
flag=True
data=[]
for icfg,cfg in enumerate(cfgs):
    print(f'{icfg}/{len(cfgs)}',end='           \r')
    infile=f'{inpath}/{cfg}/discNJN_jg;stout.h5'
    with h5py.File(infile) as f:
        if flag:
            moms=f['moms'][:]
            dic={}
            for i,mom in enumerate(moms):
                dic[tuple(mom)]=i
            
            moms_target_avg=[list(mom) for mom in moms if mom[3]**2+mom[4]**2+mom[5]**2==2]
            inds=[dic[tuple(mom)] for mom in moms_target_avg]
            flag=False       
            
        # print(f['data']['N_N_jg;stout20_6'])
        t=f['data']['N_N_jg;stout20_6'][:,inds,:,:]
        data.append(t)
    # break
data=np.array(data)
c3pt_avg=yu.jackknife(data)
print(c3pt_avg.shape)

725
(725, 7, 12, 4, 10)
(725, 7, 1, 4, 10)


In [114]:
ens='b'
mNs=ens2mN[ens]    
mpirs=[(mom,proj,insert,ri) for mom in moms_target for proj in projs for insert in inserts for ri in [0,1] if insert[0]!=insert[1] and useQ_noAvg(mom,proj,insert)[ri]]

# path='/p/project1/ngff/kummer3/disconnected_new/masses.pkl'
# with open(path,'rb') as f:
#     t=pickle.load(f)
#     mNs=t['bins'][:]
    

# pirs=[]
# path='/p/project1/ngff/kummer3/disconnected_new/required_momenta'
# with open(path) as f:
#     t=f.read().splitlines()
#     for ele in t:
#         _,ps,d,p,r=ele.split(', ')
#         ps=ps.split('(')[-1]
#         mom=[0,0,0,int(ps[0:2]),int(ps[3:5]),int(ps[6:8])]
#         pir=(mom,p[-2:],d[-2:],{'real':0,'imag':1}[r[-4:]])
#         pirs.append(pir)
        
dic={}
for i,mom in enumerate(moms_target):
    dic[tuple(mom)]=i
L=64
G=np.array([[funcs_ri[ri](ME2FF(m,np.array(mom[3:])*(2*np.pi/L),[0,0,0],proj,insert)) for mom,proj,insert,ri in mpirs] for m in mNs])
G_avg=np.array([[funcs_ri[ri](ME2FF(m,np.array([0,1,1])*(2*np.pi/L),[0,0,0],proj,insert)) for proj,insert,ri in pirs] for m in mNs])

inpath=f'/p/project1/ngff/li47/code/scratch/run/02_discNJN_1D_run3/{ens2full[ens]}_backup/data_merge/data.h5'
with h5py.File(inpath) as f:
    moms=f['N.h5/moms'][:]
    imoma=0; imomb=10
    print(moms[imoma],moms[imomb])
    print()
    cNa=f['N.h5/data/N_N'][:,:,imoma]
    cNb=f['N.h5/data/N_N'][:,:,imomb]
    cNa=yu.jackknife(np.real(cNa))
    cNb=yu.jackknife(np.real(cNb))

tf=6
ratio=c3pt_0avg/np.sqrt(
        cNa[:,tf:tf+1]*cNb[:,tf:tf+1]*\
        cNa[:,:tf+1][:,::-1]/cNa[:,:tf+1]*\
        cNb[:,:tf+1]/cNb[:,:tf+1][:,::-1]
)[:,:,None,None,None]

M_all=np.transpose([funcs_ri[ri](ratio[:,:,dic[tuple(mom)],projs.index(proj),inserts.index(insert)]) for mom,proj,insert,ri in mpirs],[1,2,0])

[0 0 0] [0 1 1]



In [150]:
def doSVD(M,G):
    Njk=len(M)
    
    cov=yu.jackmec(M)[-1]
    cov=np.diag(np.diag(cov))
    covI=np.linalg.inv(cov)
    covIsq = sqrtm(covI)
    def get(g,m):
        gt=covIsq@g
        u,s,vT=np.linalg.svd(gt)
        sI=np.zeros(gt.T.shape)
        np.fill_diagonal(sI,1/s)
        return vT.T@sI@(u.T)@covIsq@m
    F=np.array([get(g[:,:],m) for g,m in zip(G,M)])
    
    return F

from itertools import permutations
from sympy.combinatorics import Permutation
elements=[(sx,sy,sz,xyz) for sx in [-1,1] for sy in [-1,1] for sz in [-1,1] for xyz in permutations([0, 1, 2], 3)]
projs=['P0','Px','Py','Pz']
projs_rotate=['Px','Py','Pz','P0']; lorentzs=['x','y','z','t']
def rotate(e,mpir):
    sx,sy,sz,xyz=e; ix,iy,iz=xyz; iix,iiy,iiz=tuple([ix,iy,iz].index(i) for i in range(3))
    sxyzt=[sx,sy,sz,1]
    xyzt=[ix,iy,iz,3]
    m,p,i,r=mpir
    m=[0,0,0,sx*m[iix+3],sy*m[iiy+3],sz*m[iiz+3]]
    
    det=sx*sy*sz*(1 if Permutation(xyz).is_even else -1)
    
    sign=1
    p=projs_rotate[xyzt[projs_rotate.index(p)]]; sign *= sxyzt[projs_rotate.index(p)]; sign *= (1 if p=='P0' else det)
    i=lorentzs[xyzt[lorentzs.index(i[0])]]+lorentzs[xyzt[lorentzs.index(i[1])]]; i=i if i[0]<=i[1] else i[1]+i[0]; sign *= (sxyzt[lorentzs.index(i[0])]*sxyzt[lorentzs.index(i[1])])
    
    mpir=(m,p,i,r)
    return sign,mpir

def doAvg(M):
    t=[]
    for ii in range(len(pirs)):
        pir=pirs[ii]
        p,i,r=pir
        mpir=([0,0,0,0,1,1],p,i,r)
        
        tt=[]
        for e in elements:
            sign,mpir_new=rotate(e,mpir)
            inew=mpirs.index(mpir_new)
            
            tt.append(sign*M[:,inew])
        tt=np.array(tt)
        tt=np.mean(tt,axis=0)
        t.append(tt)
    t=np.transpose(t)
    return t


M_all_0avg=M_all[:,3,:]
M_all_avg=doAvg(M_all_0avg)
# print(M_all_avg[0])

F_0avg=doSVD(M_all_0avg,G)
F_avg=doSVD(M_all_avg,G_avg)

mean,err=yu.jackme(F_0avg)
print('SVD on 132 equations:')
print('[A20,B20,C20]=',[yu.un2str(m,e,3) for m,e in zip(mean,err)])
mean,err=yu.jackme(F_avg)
print('SVD on 6 averaged equations:')
print('[A20,B20,C20]=',[yu.un2str(m,e,3) for m,e in zip(mean,err)])

SVD on 132 equations:
[A20,B20,C20]= ['0.3346(821)', '-0.0865(831)', '-0.443(232)']
SVD on 6 averaged equations:
[A20,B20,C20]= ['0.3391(824)', '-0.0898(833)', '-0.467(233)']


In [142]:
# SVD test

from scipy.linalg import sqrtm
funcs_ri=[np.real,np.imag]

name='_unequal'

def standarizeMom(mom):
    t=np.abs(mom)
    t.sort()
    return t

FFs=['A20','B20','C20']
def run(ens):
    inpath=f'/p/project1/ngff/li47/code/scratch/run/02_discNJN_1D_run3/{ens2full[ens]}_backup/data_merge/data.h5'
    outpath=f'/p/project1/ngff/li47/code/scratch/run/02_discNJN_1D_run3/{ens2full[ens]}_backup/data_merge/data_SVD{name}.h5'
    with h5py.File(inpath) as f, h5py.File(outpath,'w') as fw:
        moms_N=f['N.h5/moms'][:]
        dic_N={}
        for i,mom in enumerate(moms_N):
            dic_N[tuple(mom)]=i
        cN=f['N.h5/data/N_N'][:]
        cN=yu.jackknife(cN)
        
        file='discNJN_jg;stout.h5'
        moms_3pt=f[file]['moms'][:]
        keys=list(f[f'{file}/data'].keys()); keys.sort()
        keys=['N_N_jg;stout20_6']
        for ikey,key in enumerate(keys):
            # if ikey<294:
            #     continue
            _,_,j,tf=key.split('_')
            if j=='jv1':
                continue
            # if j!='jq;stout10':
            #     continue
            tf=int(tf)
            # print(ens,f'{ikey}/{len(keys)}',key,end='              \r')
            data=f[f'{file}/data'][key][:]
            
            for imom,mom in enumerate(moms_3pt):
                if not np.all(mom==[0,0,0,0,1,1]):
                    continue
                mom_str='_'.join([str(ele) for ele in mom])
                # print(imom,len(moms_3pt),mom_str)
                pa=mom[:3]; q=mom[3:6]; pb=pa+q
                
                cNa=cN[:,:,dic_N[tuple(standarizeMom(pa))]]
                cNb=cN[:,:,dic_N[tuple(standarizeMom(pb))]]
                Njk=len(cNa)
                
                L=ens2N[ens]
                n1vec=np.array(mom[:3]); nqvec=np.array(mom[3:6])
                nvec=n1vec+nqvec
                pvec=nvec*(2*np.pi/L); p1vec=n1vec*(2*np.pi/L)
                qvec=nqvec*(2*np.pi/L)
                
                xE_jk=np.sqrt(pvec.dot(pvec)+ens2mN[ens]**2)
                xE1_jk=np.sqrt(p1vec.dot(p1vec)+ens2mN[ens]**2)
                Q2_jk=(qvec.dot(qvec) - (xE_jk-xE1_jk)**2 )
                Q2=np.mean(Q2_jk)
                
                c3pt=data[:,:,imom,:,:]
                c3pt=yu.jackknife(c3pt)
                # print(f'{j};mom={mom};ts=6,tins=3;i_jk=0')
                ratio=c3pt/np.sqrt(
                        cNa[:,tf:tf+1]*cNb[:,tf:tf+1]*\
                        cNa[:,:tf+1][:,::-1]/cNa[:,:tf+1]*\
                        cNb[:,:tf+1]/cNb[:,:tf+1][:,::-1]
                )[:,:,None,None]
                
                pirs=[(proj,insert,ri) for proj in projs for insert in inserts for ri in [0,1] if insert[0]!=insert[1] and useQ(mom,proj,insert)[ri]]
                # pirs=[(proj,insert,ri) for proj in projs for insert in inserts for ri in [0,1] if useQ(mom,proj,insert)[ri]]
                G=np.array([[funcs_ri[ri](ME2FF(m,pvec,p1vec,proj,insert)) for proj,insert,ri in pirs] for m in ens2mN[ens]])
                
                # print('components:')
                # print(pirs)
                # print('decomposition w/o dividing error:')
                # print(G[0])
                if len(G[0])==0:
                    rank=0
                else:
                    U, S, VT = np.linalg.svd(G[0])
                    tol = 1e-10
                    rank = np.sum(S > tol)
                if rank == 3 or (rank==1 and np.all(q==[0,0,0])):
                    M_all=np.transpose([funcs_ri[ri](ratio[:,:,projs.index(proj),inserts.index(insert)]) for proj,insert,ri in pirs],[1,2,0])
                    # print(M_all.shape)
                    # print('matrix elements:')
                    # print(M_all[0,3])
                    
                    t=M_all[:,3]
                    print(t[0])
                    
                    
                    t=np.zeros([Njk,tf+1,rank])
                    for tc in range(tf+1):
                        M=M_all[:,tc]
                        cov=yu.jackmec(M)[-1]
                        cov=np.diag(np.diag(cov))
                        covI=np.linalg.inv(cov)
                        covIsq = sqrtm(covI)
                        def get(g,m):
                            gt=covIsq@g
                            u,s,vT=np.linalg.svd(gt)
                            sI=np.zeros(gt.T.shape)
                            np.fill_diagonal(sI,1/s)
                            return vT.T@sI@(u.T)@covIsq@m
                        F=np.array([get(g[:,:rank],m) for g,m in zip(G,M)])
                        
                        if tc==3:
                            # print('inverse err:')
                            # print(np.diag(covIsq))
                            # print('decomposition divided by errors:')
                            # print(covIsq@G[0])
                            # print('combinations')
                            gt=covIsq@G[0]
                            u,s,vT=np.linalg.svd(gt)
                            sI=np.zeros(gt.T.shape)
                            np.fill_diagonal(sI,1/s)
                            tt=vT.T@sI@(u.T)@covIsq
                            # print(tt[1])
                            # print(M_all[0,3]*tt[1])
                        
                        t[:,tc,:]=F
                    for i in range(t.shape[-1]):
                        FF=FFs[i]
                        key=f'{FF}_{j}/{mom_str}/{tf}'
                        if key in fw:
                            del fw[key]
                        fw.create_dataset(key,data=t[:,:,i])
                    # print('A20,B20,C20:')
                    # print(t[0,3])
                    # print(t[0,:])
                    # print('mean,err')
                    # print(t.shape)
                    print(yu.jackme(t[:,3]))
        
run('b')

[ 0.01355997 -0.0093984   0.00600327 -0.00628641 -0.00014505  0.00015421]
(array([ 0.33911789, -0.08980573, -0.46681715]), array([0.08235209, 0.08329881, 0.23258682]))


In [85]:
# check if error comparable
# check if sign agrees with decomposition
# check if mean agrees within n-sigma

from itertools import permutations
from sympy.combinatorics import Permutation
elements=[(sx,sy,sz,xyz) for sx in [-1,1] for sy in [-1,1] for sz in [-1,1] for xyz in permutations([0, 1, 2], 3)]
projs=['P0','Px','Py','Pz']
projs_rotate=['Px','Py','Pz','P0']; lorentzs=['x','y','z','t']
def rotate(e,mpir):
    sx,sy,sz,xyz=e; ix,iy,iz=xyz; iix,iiy,iiz=tuple([ix,iy,iz].index(i) for i in range(3))
    sxyzt=[sx,sy,sz,1]
    xyzt=[ix,iy,iz,3]
    m,p,i,r=mpir
    m=[0,0,0,sx*m[iix+3],sy*m[iiy+3],sz*m[iiz+3]]
    
    det=sx*sy*sz*(1 if Permutation(xyz).is_even else -1)
    
    sign=1
    p=projs_rotate[xyzt[projs_rotate.index(p)]]; sign *= sxyzt[projs_rotate.index(p)]; sign *= (1 if p=='P0' else det)
    i=lorentzs[xyzt[lorentzs.index(i[0])]]+lorentzs[xyzt[lorentzs.index(i[1])]]; i=i if i[0]<=i[1] else i[1]+i[0]; sign *= (sxyzt[lorentzs.index(i[0])]*sxyzt[lorentzs.index(i[1])])
    
    mpir=(m,p,i,r)
    return sign,mpir

for i in range(len(mpirs)):
    # break
    mpir=mpirs[i]
    mean,err=yu.jackme(M_all[:,3,i])
    mean0,err0=mean,err
    
    decomp0=G[0,i]
    
    # print(mpir,yu.un2str(mean,err,forceResult=1), decomp0,end=': ')
    print(mpir,yu.un2str(mean,err,forceResult=1),end=': ')
    for e in elements:
        sign,mpir_new=rotate(e,mpir)
        
        inew=mpirs.index(mpir_new)
        mean,err=yu.jackme(sign*M_all[:,3,inew])
        if np.abs(err-err0)/err0>0.2:
            print(yu.un2str(mean,err,forceResult=1),end=', ')
        if np.abs(mean-mean0)/err0>3:
            print(yu.un2str(mean,err,forceResult=1),end=', ')
            
        decomp=sign*G[0,inew]
        dd=np.sum(np.abs(decomp-decomp0)/np.abs(decomp0) if decomp0[2]!=0 and decomp[2]!=0 else (np.abs(decomp-decomp0)/np.abs(decomp0))[:2])
        if dd!=0:
            print(dd,end=',')
    print()

([0, 0, 0, -1, -1, 0], 'P0', 'tx', 1) -0.011(14): 
([0, 0, 0, -1, -1, 0], 'P0', 'ty', 1) -0.031(14): 
([0, 0, 0, -1, -1, 0], 'P0', 'xy', 0) -0.025(14): 
([0, 0, 0, -1, -1, 0], 'Px', 'tz', 0) -0.0018(29): -0.0120(30), -0.0109(30), -0.0120(30), -0.0109(30), 
([0, 0, 0, -1, -1, 0], 'Px', 'xz', 1) -0.0030(28): 
([0, 0, 0, -1, -1, 0], 'Px', 'yz', 1) 0.0019(28): 
([0, 0, 0, -1, -1, 0], 'Py', 'tz', 0) 0.0085(29): 
([0, 0, 0, -1, -1, 0], 'Py', 'xz', 1) 0.0026(28): 
([0, 0, 0, -1, -1, 0], 'Py', 'yz', 1) -0.0027(27): 
([0, 0, 0, -1, -1, 0], 'Pz', 'tx', 0) 0.0033(29): 
([0, 0, 0, -1, -1, 0], 'Pz', 'ty', 0) -0.0049(29): 
([0, 0, 0, -1, 0, -1], 'P0', 'tx', 1) -0.008(13): 
([0, 0, 0, -1, 0, -1], 'P0', 'tz', 1) 0.005(13): -0.038(13), -0.038(13), 
([0, 0, 0, -1, 0, -1], 'P0', 'xz', 0) -0.015(15): 
([0, 0, 0, -1, 0, -1], 'Px', 'ty', 0) 0.0109(30): -0.0001(29), -0.0001(29), 0.0006(30), 0.0018(29), 0.0006(30), 0.0018(29), 
([0, 0, 0, -1, 0, -1], 'Px', 'xy', 1) -0.0015(27): 
([0, 0, 0, -1, 0, -1], 'Px', '

In [98]:
# check average rotations

from scipy.linalg import sqrtm
funcs_ri=[np.real,np.imag]

name='_unequal'

def standarizeMom(mom):
    t=np.abs(mom)
    t.sort()
    return t

dic={}
for i,mom in enumerate(moms_target):
    dic[tuple(mom)]=i

inpath=f'/p/project1/ngff/li47/code/scratch/run/02_discNJN_1D_run3/{ens2full[ens]}_backup/data_merge/data.h5'
with h5py.File(inpath) as f:
    moms_N=f['N.h5/moms'][:]
    dic_N={}
    for i,mom in enumerate(moms_N):
        dic_N[tuple(mom)]=i
    cN=f['N.h5/data/N_N'][:]
    cN=yu.jackknife(cN)
    
    file='discNJN_jg;stout.h5'
    moms_3pt=[list(mom) for mom in f[file]['moms'][:]]
    keys=list(f[f'{file}/data'].keys()); keys.sort()
    keys=['N_N_jg;stout20_6']
    
    mom=[0,0,0,0,1,1]
    t=f[f'{file}/data/{keys[0]}'][:,:,moms_3pt.index(mom)]
    c3pt=yu.jackknife(t)
    
    # print(c3pt_avg.shape)
    # print(c3pt.shape)
    # t=c3pt-c3pt_avg[:,:,0]
    # t=np.sum(np.abs(t))
    # print(t)
    
    mom=np.array(mom)
    pa=mom[:3]; q=mom[3:6]; pb=pa+q
                
    cNa=cN[:,:,dic_N[tuple((pa))]]
    cNb=cN[:,:,dic_N[tuple((pb))]]
    Njk=len(cNa)
    
    L=64
    n1vec=np.array(mom[:3]); nqvec=np.array(mom[3:6])
    nvec=n1vec+nqvec
    pvec=nvec*(2*np.pi/L); p1vec=n1vec*(2*np.pi/L)
    qvec=nqvec*(2*np.pi/L)
    
    xE_jk=np.sqrt(pvec.dot(pvec)+ens2mN[ens]**2)
    xE1_jk=np.sqrt(p1vec.dot(p1vec)+ens2mN[ens]**2)
    Q2_jk=(qvec.dot(qvec) - (xE_jk-xE1_jk)**2 )
    Q2=np.mean(Q2_jk)
    
    ratio=c3pt/np.sqrt(
            cNa[:,tf:tf+1]*cNb[:,tf:tf+1]*\
            cNa[:,:tf+1][:,::-1]/cNa[:,:tf+1]*\
            cNb[:,:tf+1]/cNb[:,:tf+1][:,::-1]
    )[:,:,None,None]
    
    pirs=[(proj,insert,ri) for proj in projs for insert in inserts for ri in [0,1] if insert[0]!=insert[1] and useQ(mom,proj,insert)[ri]]
    M_all_avg=np.transpose([funcs_ri[ri](ratio[:,:,projs.index(proj),inserts.index(insert)]) for proj,insert,ri in pirs],[1,2,0])
    
    M_all_3pt_0avg=np.transpose([funcs_ri[ri](c3pt_0avg[:,:,dic[tuple(mom)],projs.index(proj),inserts.index(insert)]) for mom,proj,insert,ri in mpirs],[1,2,0])
    M_all_3pt_avg=np.transpose([funcs_ri[ri](c3pt_avg[:,:,0,projs.index(proj),inserts.index(insert)]) for proj,insert,ri in pirs],[1,2,0])
    
    mom=[0,0,0,0,1,1]
    for ii in range(len(pirs)):
        pir=pirs[ii]
        p,i,r=pir
        mpir=(mom,p,i,r)
        
        t=M_all_avg[:,3,ii]
        mean0,err0=yu.jackme(t)
        t=[]
        for e in elements:
            sign,mpir_new=rotate(e,mpir)
            inew=mpirs.index(mpir_new)
            
            t.append(sign*M_all[:,3,inew])
        t=np.transpose(t)
        t=np.mean(t,axis=1)
        mean,err=yu.jackme(t)
        print(pir,yu.un2str(mean0,err0,forceResult=1),yu.un2str(mean,err,forceResult=1))
        

        t=M_all_3pt_avg[:,3,ii]
        mean0,err0=yu.jackme(t)
        t=[]
        for e in elements:
            sign,mpir_new=rotate(e,mpir)
            inew=mpirs.index(mpir_new)
            
            t.append(sign*M_all_3pt_0avg[:,3,inew])
        t=np.transpose(t)
        t=np.mean(t,axis=1)
        mean,err=yu.jackme(t)
        print(pir,yu.un2str(mean0,err0),yu.un2str(mean,err))
        
        print()
        # break
    
    

('P0', 'tz', 1) 0.0136(37) 0.0136(37)
('P0', 'tz', 1) 2.35(65)e-11 2.35(65)e-11

('P0', 'yz', 0) -0.0096(56) -0.0096(56)
('P0', 'yz', 0) -1.65(97)e-11 -1.65(97)e-11

('Px', 'tz', 0) 0.00596(60) 0.00596(60)
('Px', 'tz', 0) 1.03(10)e-11 1.03(10)e-11

('Pz', 'tx', 0) -0.00629(62) -0.00629(62)
('Pz', 'tx', 0) -1.08(11)e-11 -1.08(11)e-11

('Pz', 'xy', 1) -0.00010(61) -0.00010(61)
('Pz', 'xy', 1) -2(10)e-13 -2(10)e-13

('Pz', 'xz', 1) 0.00020(61) 0.00020(61)
('Pz', 'xz', 1) 3(10)e-13 3(10)e-13



In [ ]:
# SVD_0avg vs SVD_avg

